Primeiro, instalamos e importamos todas as bibliotecas:

In [ ]:
%%capture
!pip install pyopencl meshio plotly==4.12.0 gmsh_api # for PC pip install gmsh
#!apt-get install gmsh
!pip3 install numba==0.51.2

import sys
#!git clone https://github.com/live-clones/gmsh
#sys.path.append('gmsh')
!git clone https://github.com/MuriloCardosoSoares/sea.git 
sys.path.append('sea')
!git clone https://github.com/bempp/bempp-cl.git _bempp-cl
sys.path.append('_bempp-cl')

import numpy as np
import pandas as pd

from sea import definitions
from sea import materials
from sea import room

Criamos o objeto da sala a ser simulada:

In [ ]:
my_room = room.Room(room_name = "my_room")

Ajustamos o vetor de frequências para as quais a simulação será feita:


In [ ]:
my_room.algorithm_control(20, 290, 1)

print(my_room.frequencies)

Simulation algotithm will run from 20 Hz up to 290 Hz. To see all frequencies, use print(self.freq_vec). 



In [ ]:
print(my_room.frequencies.freq_vec)

[ 20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37
  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55
  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73
  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91
  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109
 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127
 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145
 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163
 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181
 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199
 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217
 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235
 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253
 254 255 256 257 258 259 260 261 262 263 264 265 26

Por padrão, a velocidade de propagação do som no ar é $c_0 = 343 \text{ m/s}$ e a densidade do ar é $\rho_0 = 1.21 \text{ kg/m}^3$. A temperatura é $T = 20^\circ \text{C}$, a umidade do ar é $\text{UR} = 50 \text{%}$ e a pressão atmosférica é $p_{atm} = 101325 \text{ Pa}$.

In [ ]:
print(my_room.air)

Air sound speed = 343.00 m/s | Air density = 1.21 kg/m^3 | Temperature = 20.00 C | Humid = 50.00 % | Atmospheric pressure = 101325.00 Pa 



Podemos ajustar as propriedades do ar que serão consideradas na simulação. Os valores de $c_0$ e $\rho_0$ podem ser definidos diretamente utilizando alterando os atributos do objeto referentes a essas variáveis:

```
my_room.air.c0 = 
my_room.air.rho0 = 
```

Porém , também é possível calcular esses valores a partir da temperatura, umidade do ar e pressão atmosférica:

In [ ]:
my_room.air.temperature = 20
my_room.air.humid = 60
my_room.air.p_atm  = 101000

my_room.air.standardized_c0_rho0()

print(my_room.air)

Air sound speed = 344.54 m/s | Air density = 1.19 kg/m^3 | Temperature = 20.00 C | Humid = 60.00 % | Atmospheric pressure = 101000.00 Pa 



Adicionamos o arquivo .geo com a geometria da sala:

In [ ]:
my_room.add_geometry()

Saving CR2_geometry.geo to CR2_geometry.geo


Definimos os materiais que irão compor as superfícies da sala. Os materiais devem ser adicionados de acordo com os *physical groups* estabelecidos no arquivo da geometria.
A primeira opção é definir diretamente a admitância, impedância de superfície ou impedância de superfície normalizada:



```
my_room.add_material(admittance=[...])
my_room.add_material(surface_impedance=[...])
my_room.add_material(normalized_surface_impedance=[...])
```

Essa opção pode ser útil caso o comportamento do material tenha sido modelado utilizando alguma ferramenta externa. Entretanto, modelos matemáticos para os principais materiais absorvedores estão disponíveis: 


In [ ]:
my_room.add_material(parameters=[0.025, 0.0025, 0.05, 0.25, 20000, 0.04], absorber_type = "perforated panel")

É possível também calcular a admitância dos materiais a partir dos coeficientes de absorção em bandas de oitava ou terço de oitava. Uma primeira opção é utilizar a regra dos 55$^\circ$, o que fará com que sejam obtidos valores reais de admitância.

In [ ]:
my_room.add_material(octave_bands_statistical_alpha = [0.8, 0.7, 0.5, 0.3, 0.1, 0.05], octave_bands = [125, 250, 500, 1000, 2000, 4000])
my_room.materials[0]._55_rule()

Outra opção é utilizar um processo de otimização para obter valores complexos de admitância a partir de coeficientes de absorção

In [ ]:
my_room.add_material(octave_bands_statistical_alpha = [0.8, 0.7, 0.5, 0.3, 0.1, 0.05], octave_bands = [125, 250, 500, 1000, 2000, 4000], absorber_type = "perforated panel")

Working on the solution of the constrained optimization problem...
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.0035104330768815977
            Iterations: 125
            Function evaluations: 903
            Gradient evaluations: 123
[0.00000000e+00 7.97063610e-12 4.18701133e+00 8.93471627e-03
 2.36379538e-01]
The solution of the optimization problem leads to rmk+1 parameters equal to [0.00000000e+00 7.97063610e-12 4.18701133e+00 8.93471627e-03
 2.36379523e-01].
Impedances, admittances and everything else related to it was already calculated.



Podemos listar os materiais que foram adicionadas à simulação:

In [ ]:
my_room.list_materials()

E deletar materiais:

In [ ]:
my_room.del_materials([1])

my_room.list_materials()

Chegamos agora à inclusão das fontes e receptores. No caso das fontes sonoras, é necessário, ao menos, definir as coordenadas e caracterizar a potência. Para tanto, uma opção é definir um nível de potência sonora, $NWS$, único que será considerado para todas as frequências:

In [ ]:
nws = 97
my_room.add_source([0.931, -2.547, 1.23], nws = nws)

Também é possível definir um espectro de potência. Nesse caso, a potência da fonte sonora irá depender da frequência:

In [ ]:
power_spec = np.array([80, 84, 89, 91, 94])
bands = np.array([16,31.5,63,125,250])
my_room.add_source([0.119, 2.880, 1.23], power_spec = power_spec, bands = bands)

Por fim, é possível possível utilizar a técnica dos harmônicos esféricos. Nesse caso, não só a resposta do fonte sonora será caracterizada, mas também a sua direcionalidade.

In [ ]:
my_room.add_source([1.498, -1.974, 1.026], type = "directional", power_correction = 84.4, elevation = np.array([0.0]), azimuth = np.array([270.0]))

É possível listar as fontes sonoras que foram incluídas e deletá-las:

In [ ]:
my_room.list_sources()
my_room.del_sources([0])
my_room.list_sources()

No caso dos receptores, é necessário definir, ao menos, a posição. Nesse caso, o receptor será considerado ominidirecional e com uma resposta completamente plana:

In [ ]:
my_room.add_receiver([-1.110, -0.256, 1.230])

Também é possível utilizar a técnica dos harmônicos esféricos para considerar a direcionalidade dos receptores. Isso permite a obtenção de resultados biauriculares.

In [ ]:
my_room.add_receiver([0, 2.026, 1.230], type = "binaural", azimuth = np.array([270]))

Tanbém é possível listar os receptores que foram incluídos e deletá-los:

In [ ]:
my_room.list_receivers()
my_room.del_receivers([1])
my_room.list_receivers()

Podemos ter uma visão geral da geometria da sala com fontes e receptores posicionados:

In [ ]:
my_room.view()

Agora é só rodar :)

In [ ]:
my_room.run()

Finalizada a simulação , é possível visualizar os resultados. É possível obter a magnitude da pressão sonora (nível de pressão sonora) e a fase: 

In [ ]:
my_room.plot_spl()
my_room.plot_phase()